In [1]:
import ccxt
import time
from collections import deque
import datetime
import pandas as pd
import filter
import login
import atr
import Long_order

# 매게변수 ticker 추가 
def get_balance_coin(balances, ticker):
    bool = True
    while True:
        # for문 추가
        for i in range(1, 11):
            if ticker in balances[i]['currency']:
                balance = balances[i]['balance']
                avg_buy_price = balances[i]['avg_buy_price']
                value = (float(avg_buy_price), float(balance))
                bal_loc = int(i)
                bool = False
                break
        if bool == False:
            break
        else:
            pass
        for j in range(1, 11):
            if balances[j]['currency'] == '':
                value = (int(0), int(0))
                bal_loc = int(j)
                break
        break
    return value, bal_loc

# 매게변수 ticker 추가
def get_ticker_ma(volume, df, balances, ticker, atr_value):

    now_price = df.iloc[-1]['close']                     # 코인 현재가
    coin_check = get_balance_coin(balances, ticker) # 코인 보유 하고 있는지 체크
    avg_price = coin_check[0][0]                         # 매수 평균가
    balance = coin_check[0][1]                             # 코인 보유 개수
    # 코인의 지갑 위치값 추가
    bal_loc = coin_check[1]
    print("지갑 위치 = {}".format(bal_loc))
    # 손절매 가격
    sell_price = avg_price - (5 * atr_value)
    # 만약 현재가격이 평균가 보다 높아지면 손절매의 가격을 높인다.
    if now_price > avg_price:
        sell_price = now_price - (5 * atr_value)
    # 매수 평균가가 int 이면 매수 조건 체크 float이면 매도 조건 체크
    if type(avg_price) == int:

        # 살 가격 정하기
        # 총자산의 10퍼
        buy_price = volume / empty_count
        Long_order.long_buy_order(buy_price, balances, bal_loc, ticker, df)
    else:
        # 현재 보유 코인 수익률 계산
        buy_profit = ((now_price - avg_price)/avg_price) * 100
        profit = round(buy_profit, 2)
        if avg_price < sell_price:
            Long_order.long_sell_order(balances, bal_loc, ticker, df)
        else:
            print(f"코인명: {ticker}, 수익률: {profit}%")
            print("내가산 가격 = {}, 시장가격 = {}".format(avg_price, now_price))
            print('---------------------------------------------')

binance = login.login()
i = 0
tickers = []
ma10 = []
ma25 = []
ma50 = []
balances = []
# 시작 날짜 입력
day = datetime.datetime(2022,5,1)
day_2 = day
# 주식차트 기준 시간 입력
interval = '1h'
interval_2 = '1d'
# day = 24, minute240 = 4
time_select = 1
time_select_2 = 24
# 초기 지갑 잔고 입력
volume = 10000000

balances = ({'currency': 'dolor', 'balance': '{}'.format(volume), 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''} #1
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''} # 5
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''} # 10 실제 딕셔너리 주소
, {'currency': '', 'balance': '', 'avg_buy_price': ''})
empty_count = 9
while True:
    assets = 0
    print('---------------------------------------------')
    print(day)
    day = int(time.mktime(day.timetuple())*1000)

    # interval에 종목시간 입력
    df_2 = binance.fetch_ohlcv(symbol = 'BTC/USDT', since = day, limit= 50, timeframe = interval_2)
    df_2 = pd.DataFrame(df_2, columns = ['date', 'open', 'high', 'low', 'close', 'volume'])
    
    ma10 = df_2['close'].rolling(window=10).mean()
    df_2.insert(len(df_2.columns), "MA10", ma10)

    ma25 = df_2['close'].rolling(window=25).mean()
    df_2.insert(len(df_2.columns), "MA25", ma25)

    ma50 = df_2['close'].rolling(window=50).mean()
    df_2.insert(len(df_2.columns), "MA50", ma50)

    volume = float(balances[0]['balance'])

    if df_2.iloc[-1]['MA10'] > df_2.iloc[-1]['MA25'] and df_2.iloc[-1]['close'] > df_2.iloc[-1]['MA50']:
        for k in range(1,11):
            if balances[k]['currency'] == '':
                empty_count += 1
                start_filter = True
                break
                
        if start_filter == True:
            tickers = filter.filter(day, interval)
        start_filter = False

        # 롱 주문
        for j in range(len(tickers)):
            ticker = tickers[j]
            df = binance.fetch_ohlcv(symbol = ticker, since = day, limit= 50, timeframe = interval)
            df = pd.DataFrame(df, columns = ['date', 'open', 'high', 'low', 'close', 'volume'])
            atr_value = atr.ATR(day, interval, ticker)
            get_ticker_ma(volume, df, balances, ticker, atr_value)
            now_price = df.iloc[-1]['close']                     # 코인 현재가
            coin_check = get_balance_coin(balances, ticker) # 코인 보유 하고 있는지 체크
            balance = coin_check[0][1]                             # 코인 보유 개수
            assets += now_price * balance
        empty_count = 0

    elif df_2.iloc[-1]['MA10'] < df_2.iloc[-1]['MA25'] or df_2.iloc[-1]['close'] < df_2.iloc[-1]['MA50']:
        # 가지고 있던 코인 팔기
        for j in range(len(tickers)):
            ticker = tickers[j]
            df = binance.fetch_ohlcv(symbol = ticker, since = day, limit= 5, timeframe = interval)
            df = pd.DataFrame(df, columns = ['date', 'open', 'high', 'low', 'close', 'volume'])
            now_price = df.iloc[-1]['close']
            print('now_price = ', now_price)
            coin_check = get_balance_coin(balances, ticker)
            avg_price = coin_check[0][0]
            bal_loc = coin_check[1]
            if type(avg_price) == float:
                Long_order.long_sell_order(balances, bal_loc, ticker, df)
        
        print('기다리는중')

    assets += float(balances[0]['balance'])

    print("총 평가 자산 = {:,.0f}원".format(assets))    
    day = datetime.datetime.fromtimestamp(int(day)/1000)
    # 시간에 24시간씩 계속 추가
    day = day + datetime.timedelta(hours = time_select)
    if i == 24:
        i = 0
        day_2 = day_2 + datetime.timedelta(hours = time_select_2)
    if day == datetime.datetime(2022, 9, 1):
        break
    i += 1
    time.sleep(0.2)


---------------------------------------------
2022-05-01 00:00:00
기다리는중
총 평가 자산 = 10,000,000원
---------------------------------------------
2022-05-01 01:00:00
기다리는중
총 평가 자산 = 10,000,000원
---------------------------------------------
2022-05-01 02:00:00
기다리는중
총 평가 자산 = 10,000,000원
---------------------------------------------
2022-05-01 03:00:00
기다리는중
총 평가 자산 = 10,000,000원
---------------------------------------------
2022-05-01 04:00:00
기다리는중
총 평가 자산 = 10,000,000원
---------------------------------------------
2022-05-01 05:00:00
기다리는중
총 평가 자산 = 10,000,000원
---------------------------------------------
2022-05-01 06:00:00
기다리는중
총 평가 자산 = 10,000,000원
---------------------------------------------
2022-05-01 07:00:00
기다리는중
총 평가 자산 = 10,000,000원
---------------------------------------------
2022-05-01 08:00:00
기다리는중
총 평가 자산 = 10,000,000원
---------------------------------------------
2022-05-01 09:00:00
기다리는중
총 평가 자산 = 10,000,000원
---------------------------------------------
2022-05-01 10:

UnboundLocalError: local variable 'value' referenced before assignment

In [ ]:
empty_count = 0
for k in range(1,11):
    empty_count += 1
print(empty_count)

10
